# Ag IQ Data Cleaning & Merging

This notebook cleans the auction data and merges it with macro economic indicators to create the training dataset.

**Goal**: Create a clean, merged dataset ready for feature engineering with:
- Valid price, date, and equipment information
- Macro indicators joined by month
- 7-year training window (2018-2025)
- ~600K+ usable records


In [ ]:
# =============================================================================
# CELL 1: Setup
# =============================================================================
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Setup complete")


In [ ]:
# =============================================================================
# CELL 2: Load Raw Data
# =============================================================================
from src.data.loaders import load_all_data

print("Loading raw data...")
data = load_all_data('../data/raw')

auctions = data['auctions']
barometer = data['barometer']
diesel = data['diesel']
el_nino = data['el_nino']
futures = data['futures']
makes = data['makes']
auctioneers = data['auctioneers']

print(f"\nRaw auction records: {len(auctions):,}")


In [ ]:
# =============================================================================
# CELL 3: Create Training Dataset
# =============================================================================
from src.data.processors import create_training_dataset

# Create cleaned and merged dataset
# Using last 7 years of data (2018-2025) for training
training_df = create_training_dataset(
    auctions=auctions,
    makes=makes,
    barometer=barometer,
    diesel=diesel,
    el_nino=el_nino,
    futures=futures,
    min_date='2018-01-01',
    max_date=None,  # Use all data up to present
    min_price=1000,
    max_price=2_000_000
)

print(f"\nFinal training dataset: {len(training_df):,} records")


In [ ]:
# =============================================================================
# CELL 4: Data Quality Check
# =============================================================================
print("=" * 60)
print("DATA QUALITY CHECK")
print("=" * 60)

print(f"\nTotal records: {len(training_df):,}")
print(f"Date range: {training_df['sold_date'].min().date()} to {training_df['sold_date'].max().date()}")
print(f"Price range: ${training_df['price'].min():,.0f} to ${training_df['price'].max():,.0f}")

print("\n--- Core Fields Coverage ---")
core_fields = ['price', 'sold_date', 'year', 'hours', 'make_key', 'region']
for field in core_fields:
    if field in training_df.columns:
        coverage = training_df[field].notna().mean() * 100
        print(f"  {field:15} {coverage:5.1f}%")

print("\n--- Macro Indicators Coverage ---")
macro_fields = ['barometer', 'diesel_price', 'el_nino_phase']
for field in macro_fields:
    if field in training_df.columns:
        coverage = training_df[field].notna().mean() * 100
        print(f"  {field:15} {coverage:5.1f}%")


In [ ]:
# =============================================================================
# CELL 5: Temporal Distribution
# =============================================================================
print("\n--- Temporal Distribution ---")

# Monthly counts
monthly_counts = training_df.groupby(training_df['sold_date'].dt.to_period('M')).size()
print(f"Months covered: {len(monthly_counts)}")
print(f"Avg sales/month: {monthly_counts.mean():.0f}")
print(f"Min sales/month: {monthly_counts.min()}")
print(f"Max sales/month: {monthly_counts.max()}")

# Yearly summary
yearly = training_df.groupby(training_df['sold_date'].dt.year).agg({
    'price': ['count', 'median', 'mean']
}).round(0)
yearly.columns = ['count', 'median_price', 'mean_price']
print(f"\nYearly Summary:")
print(yearly)


In [ ]:
# =============================================================================
# CELL 6: Visualize Cleaning Results
# =============================================================================
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Price distribution
axes[0, 0].hist(training_df['price'], bins=100, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Price ($)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('Price Distribution (Cleaned Data)')
axes[0, 0].axvline(training_df['price'].median(), color='red', linestyle='--', 
                    label=f"Median: ${training_df['price'].median():,.0f}")
axes[0, 0].legend()

# Sales over time
monthly_df = training_df.groupby(training_df['sold_date'].dt.to_period('M')).size().reset_index()
monthly_df.columns = ['month', 'count']
monthly_df['month'] = monthly_df['month'].dt.to_timestamp()
axes[0, 1].plot(monthly_df['month'], monthly_df['count'], marker='.', markersize=3)
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Number of Sales')
axes[0, 1].set_title('Sales Volume Over Time (2018-2025)')

# Hours distribution
valid_hours = training_df[training_df['hours'].notna()]['hours']
axes[1, 0].hist(valid_hours, bins=100, edgecolor='black', alpha=0.7)
axes[1, 0].set_xlabel('Hours')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title(f'Hours Distribution (n={len(valid_hours):,})')
axes[1, 0].axvline(valid_hours.median(), color='red', linestyle='--',
                    label=f"Median: {valid_hours.median():,.0f}")
axes[1, 0].legend()

# Regional distribution
region_counts = training_df['region'].value_counts()
axes[1, 1].barh(range(len(region_counts)), region_counts.values)
axes[1, 1].set_yticks(range(len(region_counts)))
axes[1, 1].set_yticklabels(region_counts.index)
axes[1, 1].set_xlabel('Number of Sales')
axes[1, 1].set_title('Sales by Region')

plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# CELL 7: Inspect Merged Columns
# =============================================================================
print("=" * 60)
print("MERGED DATASET COLUMNS")
print("=" * 60)

print(f"\nTotal columns: {len(training_df.columns)}")
print("\nAll columns:")
for i, col in enumerate(sorted(training_df.columns), 1):
    dtype = training_df[col].dtype
    non_null = training_df[col].notna().sum()
    pct = non_null / len(training_df) * 100
    print(f"  {i:2}. {col:30} {str(dtype):15} {pct:5.1f}% coverage")


In [ ]:
# =============================================================================
# CELL 8: Sample Data Inspection
# =============================================================================
print("\n--- Sample Records ---")
print("\nFirst 5 records (key columns):")
key_cols = ['sold_date', 'price', 'year', 'hours', 'make_key', 'region', 
            'barometer', 'diesel_price', 'el_nino_phase']
display_cols = [c for c in key_cols if c in training_df.columns]
print(training_df[display_cols].head())

print("\n--- Summary Statistics ---")
print(training_df[['price', 'year', 'hours']].describe())


In [ ]:
# =============================================================================
# CELL 9: Save Processed Dataset
# =============================================================================
output_path = Path('../data/processed/training_data.parquet')
output_path.parent.mkdir(parents=True, exist_ok=True)

print("Saving processed dataset...")
training_df.to_parquet(output_path, index=False)

file_size_mb = output_path.stat().st_size / (1024 * 1024)
print(f"✓ Saved to: {output_path}")
print(f"  File size: {file_size_mb:.1f} MB")
print(f"  Records: {len(training_df):,}")
print(f"  Columns: {len(training_df.columns)}")

print("\n" + "=" * 60)
print("DATA CLEANING COMPLETE")
print("=" * 60)
print("\nNext step: Feature engineering (notebook 03)")
print("  - Equipment features (age, utilization)")
print("  - Temporal features (seasonality)")
print("  - Macro features (normalized indicators)")
